In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pypsa

In [2]:
def annuity(n,r): # annuity factor
    return r/(1.-1./(1.+r)**n)*(r>0)+1/n*(r<=0)

plants = pd.read_csv('data/plants/plants.csv', sep = ';', index_col = 0, comment = '#')

In [3]:
switch = {'year': 2030,
          'fuel': 'NH3',
          'battery': True,
          'wind': True,
          'solar': True,
          'increase_demand': False,
          'connection': True,
          'link_constraint': False,
          'location': 'Energy Island'}

In [4]:
# time index from year
t = pd.date_range('%d-01-01 00:00'%switch['year'], '%d-12-31 23:00'%switch['year'], freq = 'H')
# leap year
if (np.mod(switch['year'],100) != 0 and np.mod(switch['year'],4) == 0) or np.mod(switch['year'],400) == 0:
    t = t[:1416].union(t[1440:])

# weather data from refinement
cf = pd.DataFrame(index = t, columns = ['Wind','PV'])
cf['Wind'] = pd.read_csv('data/cf/wind_%s.csv'%plants['location'][switch['location']], index_col = 0).to_numpy()
cf['PV'] = pd.read_csv('data/cf/pv_%s.csv'%plants['location'][switch['location']], index_col = 0).to_numpy()

In [5]:
# efficiencies
eff = pd.Series(index = ['H2 Plant','NH3 Plant','H2 Engine','NH3 Engine','Battery','HVDC'], dtype = float)
eff['H2 Plant'] = 141.8/(50*3.6)
eff['NH3 Plant'] = 23/(9.9*3.6)
eff['H2 Engine'] = 0.55
eff['NH3 Engine'] = 0.38
eff['Battery'] = 0.9216
eff['HVDC'] = 0.97 # per 1000 km

# battery max charging hours
h_b = 6

# energy island fuel demand
load = 0.026*1.9e12/3.6/1000/8760 # MWh in every hour
if switch['increase_demand']: load *= 1.015**(switch['year']-2030)

# offwind: 21 large turbines, off-shore
# onwind: 20 Large wind turbines on land
# pv: large scale utility systems (NO axis-tracking)
# H2: AEC 100MW
# NH3: no electrolyzer, ASU ?
if switch['year'] == 2030:
    n = np.array([30,30,40,30,30,25]) # expected lifetime
    r = np.array([0.07]) # constant discount rate
    costdata = pd.DataFrame(np.array([[1.93e6,1.04e6,3.8e5,4.5e5,1.3e6,1.42e5*h_b+1.6e5], # €/MW
                                      [36053,12600,7250,9000,39000,540], # €/MW/year
                                      [2.7,1.35,0.01,0.01,0.02,0]]), # €/MWh
                            index = ['Investment','FOM','VOM'],
                            columns = ['Offshore Wind','Onshore Wind','Solar PV','H2 Plant','NH3 Plant','Battery'])
elif switch['year'] == 2040: 
    n = np.array([30,30,40,32,30,30]) # expected lifetime
    r = np.array([0.07]) # constant discount rate
    costdata = pd.DataFrame(np.array([[1.81e6,9.8e5,3.3e5,3e5,1.1e6,9.4e4*h_b+1e5], # €/MW
                                      [33169,11592,6625,6000,32000,540], # €/MW/year
                                      [2.5,1.24,0.01,0.01,0.02,0]]), # €/MWh
                            index = ['Investment','FOM','VOM'],
                            columns = ['Offshore Wind','Onshore Wind','Solar PV','H2 Plant','NH3 Plant','Battery'])
elif switch['year'] == 2050: 
    n = np.array([30,30,40,35,30,30]) # expected lifetime
    r = np.array([0.07]) # constant discount rate
    costdata = pd.DataFrame(np.array([[1.78e6,9.6e5,3e5,2.5e5,8e5,7.5e4*h_b+6e4], # €/MW
                                      [32448,11340,6250,5000,24000,540], # €/MW/year
                                      [2.4,1.22,0.01,0.01,0.02,0]]), # €/MWh
                            index = ['Investment','FOM','VOM'],
                            columns = ['Offshore Wind','Onshore Wind','Solar PV','H2 Plant','NH3 Plant','Battery'])
ccost = annuity(n,r)*costdata.loc['Investment']+costdata.loc['FOM'] # €/MW

In [6]:
dfr = 1.2 # detour factor

# onshore and offshore straight distance
if switch['location'] == 'Energy Island':
    distance = {'DK': (223,81.5), # Thorsminde-DK[Odense] km https://www.mapdevelopers.com/distance_from_to.php
                'DE': (420.5,306.2), # Cuxhaven-DE
                'NO': (441.3,200), # Kristiansand-NO
                'NL': (155,354), # Groningen-NL
                'GB': (98.28,550.3)} # Newcastle_upon_Tyne-GB
    
elif switch['location'] == 'DK Wind':
    distance = {'DK': (221.5,0), #  
                'DE': (771.14,0), #
                'NO': (466.5,182.32), # Hjørring-Larvik
                'SE': (997.69,0), # 
                'GB': (92.2,635.93)} # Thorsminde-Newcastle upon Tyne
    
elif switch['location'] == 'DK Solar':
    distance = {'DK': (104.15,0), #  
                'DE': (500,50), # Rødbyhavn-Fehmarn
                'SE': (781.66,0), # 
                'NO': (649.37,182.32), # Hjørring-Larvik
                'PL': (520,222)} # Swinoujscie
    
elif switch['location'] == 'DE Wind':
    distance = {'DK': (495.15,0), #  
                'DE': (401.42,0), # 
                'CZ': (850,0), # 
                'NL': (296.54,0), # 
                'BE': (554,0)} # 

elif switch['location'] == 'DE Solar':
    distance = {'CZ': (623,0), #  
                'DE': (516.3,0), # 
                'AT': (270.26,0), # 
                'CH': (441.79,0), # 
                'FR': (1167.67,0)} #
    
elif switch['location'] == 'NO Wind':
    distance = {'DK': (498.83,140.4), # Kristiansand-Hjørring 
                'SE': (758.43,0), # 
                'NO': (485,0), # 
                'GB': (483.61,496.4)} # Aberdeen
    
elif switch['location'] == 'NO Solar':
    distance = {'DK': (306.31,182.32), #  
                'SE': (381.37,0), # 
                'NO': (251.19,0), # 
                'GB': (237.63,708.46)} # Newcastle upon Tyne 
    
elif switch['location'] == 'NL Wind':
    distance = {'DK': (118.5,316.37), # Ribe 
                'DE': (538.41,0), # 
                'GB': (175,328.21), # Hull
                'NL': (152.32,0), # 
                'BE': (360.37,0)} # 
    
elif switch['location'] == 'NL Solar':
    distance = {'DK': (118.5,387.97), # Ribe 
                'DE': (582,0), # 
                'BE': (295.44,0), # 
                'NL': (124.44,0), # 
                'GB': (173.85,353.46)} # Hull
    
elif switch['location'] == 'GB Wind':
    distance = {'BE': (497.51,315.87), # London-Antwerp
                'IE': (378.67,254.88), # Kendal-Dublin
                'FR': (965.12,205.3), # Southampton-Caen
                'NL': (542.16,245.48), # Colchester-Rotterdam
                'GB': (87.28,0)} # 
    
elif switch['location'] == 'GB Solar':
    distance = {'FR': (593.73,50.54), # Calais
                'BE': (79.76,224.75), # Antwerp
                'IR': (758.39,108.14), # Holyhead-Dublin
                'NL': (105.82,245.15), # Rotterdam
                'GB': (505.98,0)} #

In [7]:
network = pypsa.Network()

network.set_snapshots(t)

network.add('Bus', 'Electricity')
network.add('Bus', switch['fuel'])

In [8]:
network.add('Link',
            switch['fuel']+' Plant',
            bus0 = 'Electricity',
            bus1 = switch['fuel'],
            p_nom_extendable = True,
            #p_nom_max = 1000,
            #p_nom_min = 5319.719576,
            efficiency = eff[switch['fuel']+' Plant'],
            capital_cost = ccost[switch['fuel']+' Plant'],
            marginal_cost = costdata.loc['VOM'][switch['fuel']+' Plant']*eff[switch['fuel']+' Plant'])

In [9]:
network.add('Load',
            'Fuel Demand',
            bus = switch['fuel'],
            p_set = load)

network.add('StorageUnit',
            'Free Tank',
            bus = switch['fuel'],
            cyclic_state_of_charge = True,
            p_nom_extendable = True)

In [10]:
if switch['wind']:
    windname = 'Onshore Wind' if plants['onshore'][switch['location']] else 'Offshore Wind'
    network.add('Generator',
                windname,
                bus = 'Electricity',
                p_nom_extendable = True,
                #p_nom_max = 10000,
                #p_nom_min = 5292.263671,
                capital_cost = ccost[windname],
                marginal_cost = costdata.loc['VOM'][windname],
                p_max_pu = cf['Wind'])

if switch['solar']:
    network.add('Generator',
                'Solar PV',
                bus = 'Electricity',
                p_nom_extendable = True,
                #p_nom_max = 10000 if plants['onshore'][switch['location']] else 46,
                #p_nom_min = 1139.036028,
                capital_cost = ccost['Solar PV'],
                marginal_cost = costdata.loc['VOM']['Solar PV'],
                p_max_pu = cf['PV'])

In [11]:
if switch['battery']:
    network.add('StorageUnit',
                'Battery',
                bus = 'Electricity',
                cyclic_state_of_charge = True,
                p_nom_extendable = True,
                #p_nom_max = 1000,
                #p_nom_min = 0,
                capital_cost = ccost['Battery'],
                marginal_cost = costdata.loc['VOM']['Battery'],
                efficiency_store = eff['Battery'],
                efficiency_dispatch = eff['Battery'],
                max_hours = h_b)

In [12]:
# Countries
if switch['connection']:
    mcost = pd.read_csv('data/market/price_%d.csv'%switch['year'], index_col = 0).set_index(t)
    load = pd.read_csv('data/market/load_%d.csv'%switch['year'], index_col = 0).set_index(t)
    for country in distance.keys():
        network.add('Bus', country)

        network.add('Generator',
                    'Elec_'+country,
                    bus = country,
                    p_nom_extendable = True,
                    marginal_cost = mcost[country])
        network.add('Load',
                    'Load_'+country, 
                    bus = country, 
                    p_set = load[country])

        cccost = (annuity(40,0.07)+0.02)*(400*distance[country][0]*dfr+2000*distance[country][1]*dfr+150000)
        cmcost = 0.01
        ceff = 1-(1-eff['HVDC'])*sum(distance[country])*dfr/1000
        if switch['link_constraint']:
            network.add('Link',
                        '%s to %s'%(switch['location'],country),
                        bus0 = 'Electricity',
                        bus1 = country,
                        p_nom_extendable = True,
                        p_nom_max = 1500,
                        efficiency = ceff,
                        capital_cost = cccost/2,
                        marginal_cost = cmcost)
            network.add('Link',
                        '%s to %s'%(country,switch['location']),
                        bus0 = country,
                        bus1 = 'Electricity',
                        p_nom_extendable = True,
                        p_nom_max = 1500,
                        efficiency = ceff,
                        capital_cost = cccost/2,
                        marginal_cost = cmcost)
        else:
            network.add('Bus', country+' Hub 1') # close to plant
            network.add('Bus', country+' Hub 2') # close to country

            network.add('Link',
                        '%s to %s Hub 1'%(switch['location'],country),
                        bus0 = 'Electricity',
                        bus1 = country+' Hub 1',
                        p_nom_extendable = True)
            network.add('Link',
                        '%s Hub 1 to %s'%(country,switch['location']),
                        bus0 = country+' Hub 1',
                        bus1 = 'Electricity',
                        p_nom_extendable = True,
                        efficiency = ceff,
                        marginal_cost = cmcost)
            network.add('Link',
                        '%s Hub 1 and %s Hub 2'%(country,country), 
                        bus0 = country+' Hub 1',
                        bus1 = country+' Hub 2',
                        p_nom_extendable = True,
                        p_nom_max = 1500,
                        p_min_pu = -1,
                        capital_cost = cccost) # marta
            network.add('Link',
                        '%s to %s Hub 2'%(country,country),
                        bus0 = country,
                        bus1 = country+' Hub 2',
                        p_nom_extendable = True)
            network.add('Link',
                        '%s Hub 2 to %s'%(country,country),
                        bus0 = country+' Hub 2',
                        bus1 = country,
                        p_nom_extendable = True,
                        efficiency = ceff,
                        marginal_cost = cmcost)
else:
    switch['link_constraint'] = False

In [13]:
if switch['link_constraint']:
    def add_cable_constrants(network):
        p = pypsa.linopt.get_var(network, 'Link', 'p_nom')
        pout = p.loc[['%s to %s'%(switch['location'],country) for country in distance.keys()]]
        pin = p.loc[['%s to %s'%(country,switch['location']) for country in distance.keys()]]
        lhs = pypsa.linopt.linexpr((1,pout.reset_index(drop = True)),(-1,pin.reset_index(drop = True)))
        pypsa.linopt.define_constraints(network, lhs, '=', 0, 'Cable')

    def extra_functionality(network, snapshots):
        add_cable_constrants(network)

In [14]:
network.lopf(network.snapshots,
             pyomo = False,
             extra_functionality = extra_functionality if switch['link_constraint'] else None,
             solver_name = 'gurobi')

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 3.84s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Set parameter Username
Academic license - for non-commercial use only - expires 2022-10-03
Read LP format model from file /var/folders/3f/817ykyb972j227v95ln_qw480000gn/T/pypsa-problem-bvoqyg5i.lp
Reading time = 1.94 seconds
obj: 849720 rows, 341676 columns, 1686242 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 849720 rows, 341676 columns and 1686242 nonzeros
Model fingerprint: 0x8fb1bd49
Coefficient statistics:
  Matrix range     [4e-06, 6e+00]
  Objective range  [1e-02, 2e+05]
  Bounds range     [2e+03, 2e+03]
  RHS range        [2e+03, 7e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 600118 rows and 65785 columns
Presolve time: 1.51s
Presolved: 249602 rows, 275891 columns, 853926 nonzeros

Ordering time: 0.04s

Barrier statistics:
 Dense cols : 9
 AA' NZ     : 5.605e+05
 Factor NZ  : 2.576e+06 (roughly 230 MB 

INFO:pypsa.linopf:Optimization successful. Objective value: 5.27e+10


('ok', 'optimal')

In [15]:
((sum(network.buses_t.marginal_price['Electricity']*network.links_t.p0[switch['fuel']+' Plant'])
+network.links.capital_cost[switch['fuel']+' Plant']*network.links.p_nom_opt[switch['fuel']+' Plant']
+network.links.marginal_cost[switch['fuel']+' Plant']*network.links_t.p0[switch['fuel']+' Plant'].sum())
/sum(network.loads_t.p['Fuel Demand']))

85.30660399355128

In [16]:
(network.buses_t.marginal_price[switch['fuel']].max(),network.buses_t.marginal_price[switch['fuel']].min())

(85.30660423161247, 85.30660423161244)

In [17]:
network.objective

52681398035.30189

In [18]:
if not switch['connection']: print(network.objective/sum(network.loads_t.p['Fuel Demand']))

In [19]:
network.generators.p_nom_opt # in MW

Offshore Wind     7782.898623
Solar PV            73.032931
Elec_DK           6543.456594
Elec_DE          72258.586184
Elec_NO          19764.446790
Elec_NL          18248.449754
Elec_GB          46336.880376
Name: p_nom_opt, dtype: float64

In [20]:
network.storage_units.p_nom_opt

Free Tank    434473.706148
Battery           0.000000
Name: p_nom_opt, dtype: float64

In [21]:
network.links.p_nom_opt # in MW

NH3 Plant                    2890.552254
Energy Island to DK Hub 1    1500.000000
DK Hub 1 to Energy Island    1500.000000
DK Hub 1 and DK Hub 2        1500.000000
DK to DK Hub 2               1500.000000
DK Hub 2 to DK               1500.000000
Energy Island to DE Hub 1    1500.000000
DE Hub 1 to Energy Island    1500.000000
DE Hub 1 and DE Hub 2        1500.000000
DE to DE Hub 2               1500.000000
DE Hub 2 to DE               1500.000000
Energy Island to NO Hub 1    1500.000000
NO Hub 1 to Energy Island    1500.000000
NO Hub 1 and NO Hub 2        1500.000000
NO to NO Hub 2               1500.000000
NO Hub 2 to NO               1500.000000
Energy Island to NL Hub 1    1500.000000
NL Hub 1 to Energy Island    1500.000000
NL Hub 1 and NL Hub 2        1500.000000
NL to NL Hub 2               1500.000000
NL Hub 2 to NL               1500.000000
Energy Island to GB Hub 1    1500.000000
GB Hub 1 to Energy Island    1500.000000
GB Hub 1 and GB Hub 2        1500.000000
GB to GB Hub 2  